## Quick notebook to take a file and separate it up into sections based on Z location

In [ ]:
import sys,os,os.path
import numpy as np
import pandas as pd
import tables as tb
import matplotlib.pyplot as plt
sys.path.append("../")
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
os.environ['ICTDIR']='/home/e78368jw/Documents/NEXT_CODE/IC'

from invisible_cities.core.core_functions   import shift_to_bin_centers
from invisible_cities.io.dst_io             import load_dst, load_dsts, df_writer

from invisible_cities.io.hits_io            import hits_from_df
from invisible_cities.reco                  import paolina_functions as plf
from invisible_cities.types.ic_types        import         types_dict_tracks
from invisible_cities.types.ic_types        import         NoneType
from invisible_cities.types.symbols         import                 HitEnergy
from invisible_cities.evm.event_model       import             HitCollection
from invisible_cities.core                  import system_of_units as units

from typing import Tuple, Callable, Optional, Sequence, Union

import core.plotting_functions as plt_func


## next done with MC signal events

In [7]:
path = '../data/CLEAN/signal_events.h5'
# find .h5 files in path
RECO_hits = load_dst(f'{path}', 'RECO', 'Events')
MC_hits = load_dst(f'{path}', 'MC', 'hits')
part_info = load_dst(f'{path}', 'MC', 'particles')
MC_tracks = load_dst(f'{path}', 'Tracking', 'MC_Tracks')
# particle data too

display(RECO_hits)
display(MC_hits)
display(part_info)
display(MC_tracks)

,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,12300006,2.460001e+13,0,6.515505,-148.908484,1,-19.125,-155.375,0.0,0.0,40.263651,7.188406,574.714427,-1.0,0.001402,-1,-1.0
1,12300006,2.460001e+13,0,6.515505,-148.908484,1,-19.125,-139.825,0.0,0.0,40.263651,5.217391,417.131443,-1.0,0.001018,-1,-1.0
2,12300006,2.460001e+13,0,6.515505,-148.908484,1,-3.575,-155.375,0.0,0.0,40.263651,15.072464,1205.046364,-1.0,0.002943,-1,-1.0
3,12300006,2.460001e+13,0,6.515505,-148.908484,1,-3.575,-139.825,0.0,0.0,40.263651,9.449275,755.471336,-1.0,0.001847,-1,-1.0
4,12300006,2.460001e+13,0,6.515505,-148.908484,1,10.975,-155.375,0.0,0.0,40.263651,8.811594,704.488618,-1.0,0.001721,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420832,12204085,2.440817e+13,1,200.640000,-7.964710,1,288.875,-16.925,0.0,0.0,1028.999786,6.608695,398.112332,-1.0,0.001020,-1,-1.0
420833,12204085,2.440817e+13,1,200.640000,-7.964710,1,304.425,-16.925,0.0,0.0,1028.999786,5.565217,335.252499,-1.0,0.000870,-1,-1.0
420834,12204085,2.440817e+13,1,200.640000,-7.964710,1,257.775,-16.925,0.0,0.0,1032.469781,5.855072,1413.585331,-1.0,0.003604,-1,-1.0
420835,12204085,2.440817e+13,1,200.640000,-7.964710,1,257.775,-1.375,0.0,0.0,1032.469781,5.623188,1357.601767,-1.0,0.003466,-1,-1.0


,event,particle_id,hit_id,X,Y,Z,time,E,label,npeak
0,12300006,32,0,36.513824,-125.199722,105.147636,2.834566,0.000001,ACTIVE,1
1,12300006,31,0,36.547150,-125.759644,104.332382,2.629001,0.002518,ACTIVE,1
2,12300006,31,1,36.605511,-126.437218,103.599846,2.633066,0.002213,ACTIVE,1
3,12300006,31,2,36.760361,-127.064156,102.854980,2.637081,0.002263,ACTIVE,1
4,12300006,31,3,37.286903,-127.554703,102.174568,2.641119,0.000403,ACTIVE,1
...,...,...,...,...,...,...,...,...,...,...
504869,12204085,69,52,209.448700,-34.775208,997.616333,2.203594,0.002556,ACTIVE,1
504870,12204085,69,53,209.446777,-34.771420,997.600708,2.203996,0.002089,ACTIVE,1
504871,12204085,69,54,209.446533,-34.771984,997.601624,2.204033,0.000024,ACTIVE,1
504872,12204085,69,55,209.445343,-34.767570,997.604004,2.204250,0.002396,ACTIVE,1


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
0,12300006,1,Tl208,True,0,398.747284,401.567871,563.659546,0.000000,398.747284,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,none,RadioactiveDecay
1,12300006,4,e-,False,1,398.747284,401.567871,563.659546,0.000762,398.747589,...,-0.017302,-0.296512,-0.155331,-0.000000,0.000000,0.000000,0.100120,2.790027e-02,RadioactiveDecay,NoProcess
2,12300006,3,anti_nu_e,False,1,398.747284,401.567871,563.659546,0.000762,-9720.689453,...,-0.919971,-0.797942,-1.187634,-0.919971,-0.797942,-1.187634,1.701039,1.871097e+04,RadioactiveDecay,Transportation
3,12300006,2,Pb208[3197.711],False,1,398.747284,401.567871,563.659546,0.000762,398.747284,...,0.937273,1.094454,1.342965,0.000000,0.000000,0.000000,0.000010,0.000000e+00,RadioactiveDecay,RadioactiveDecay
4,12300006,6,gamma,False,2,398.747284,401.567871,563.659546,0.000762,398.441864,...,-0.275587,-0.117285,0.500404,-0.000000,-0.000000,0.000000,0.583188,6.463141e-01,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49929,12204085,58991,e-,False,58504,246.444626,434.608490,1265.968140,5.705839,246.519730,...,-0.134671,0.556736,0.245645,-0.000000,0.000000,0.000000,0.294949,4.999868e-01,compt,NoProcess
49930,12204085,69,e-,False,66,208.218964,-34.900761,997.138367,2.151887,209.444992,...,-0.233796,-0.094158,-0.059741,-0.000000,0.000000,0.000000,0.061902,6.112734e+00,conv,NoProcess
49931,12204085,68,e-,False,66,384.466278,339.716492,633.618896,0.314111,384.466278,...,0.004220,0.003004,-0.002740,0.000000,0.000000,-0.000000,0.000034,6.015512e-08,compt,NoProcess
49932,12204085,67,e-,False,66,384.466278,339.716492,633.618896,0.314111,384.484070,...,0.379871,-0.092510,0.356779,0.000000,-0.000000,-0.000000,0.224713,1.074977e-01,compt,NoProcess


,event,trackID,energy,length,numb_of_voxels,numb_of_hits,numb_of_tracks,x_min,y_min,z_min,...,blob1_z,blob2_x,blob2_y,blob2_z,eblob1,eblob2,ovlp_blob_energy,vox_size_x,vox_size_y,vox_size_z
0,1212003,0,1.551080,497.286267,511,967,2,155.699524,227.459152,561.196777,...,576.539307,194.930542,238.221573,672.626404,0.292241,0.236851,0.000000,0.996020,0.997924,0.997965
1,1212003,1,0.041426,0.000000,1,82,2,211.962662,185.492798,696.540649,...,696.766026,212.022651,185.602417,696.766026,0.041426,0.041426,0.041426,0.996020,0.997924,0.997965
2,1212019,0,1.062401,374.265850,352,649,1,83.538750,139.741592,574.431152,...,628.451660,216.694138,242.315384,574.695740,0.233463,0.184444,0.000000,0.999854,0.996397,0.991111
3,1212037,0,1.592507,626.026621,597,972,5,194.850876,74.949455,230.235168,...,358.229431,341.690918,205.188858,240.594391,0.233390,0.224443,0.000000,0.999774,0.998009,0.999099
4,1212037,1,0.116812,7.554383,14,106,5,387.340179,294.523743,634.873535,...,635.005979,390.522777,294.691948,641.010558,0.116812,0.116812,0.116812,0.999774,0.998009,0.999099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1355,12336087,0,1.592507,548.400899,536,779,3,198.715042,115.408508,475.540802,...,538.499878,207.464091,171.687855,488.056352,0.194041,0.193727,0.000000,0.995961,0.998120,0.998279
1356,12336087,1,0.140402,4.865310,14,118,3,276.328186,208.832825,833.028992,...,835.246704,279.190606,210.913630,833.314586,0.140402,0.140402,0.140402,0.995961,0.998120,0.998279
1357,12336087,2,0.090864,3.723881,10,62,3,258.667389,196.071686,805.258484,...,805.933119,260.316284,196.071686,806.529724,0.090864,0.090864,0.090864,0.995961,0.998120,0.998279
1358,12336095,0,1.592507,481.136755,536,927,2,223.911438,194.934479,461.894897,...,550.468201,254.417557,257.600708,479.306793,0.324534,0.195551,0.000000,0.997759,0.997815,0.998575


In [32]:
z_min = MC_hits['Z'].min()
z_max = MC_hits['Z'].max()

print(f"Minimum Z: {z_min}, Maximum Z: {z_max}")



Minimum Z: -6.436517985264345e-14, Maximum Z: 1448.538330078125


In [33]:


# Define multiple regions with minimum and maximum z values
z_regions = [(0, 400), (401, 800), (801, 1200), (1201, 1600)]

# Create a dictionary to store filtered events for each region
filtered_events_by_region = {}

# Loop through each region and filter the events
for i, (z_min, z_max) in enumerate(z_regions):
    filtered_events_by_region[f"Region_{i+1}: {z_min, z_max}"] = MC_hits[(MC_hits['Z'] >= z_min) & (MC_hits['Z'] <= z_max)]

# Display the filtered events for each region
for region, events in filtered_events_by_region.items():
    print(f"{region}:")
    display(events)

    # Separate each of the other dataframes into the defined regions based on event or event_id
    filtered_RECO_hits_by_region = {}
    filtered_MC_tracks_by_region = {}
    filtered_part_info_by_region = {}

    for i, (z_min, z_max) in enumerate(z_regions):
        region_key = f"Region_{i+1}: {z_min, z_max}"
        events_in_region = filtered_events_by_region[region_key]['event'].unique()
        
        filtered_RECO_hits_by_region[region_key] = RECO_hits[RECO_hits['event'].isin(events_in_region)]
        filtered_MC_tracks_by_region[region_key] = MC_tracks[MC_tracks['event'].isin(events_in_region)]
        filtered_part_info_by_region[region_key] = part_info[part_info['event_id'].isin(events_in_region)]

    # Display the filtered data for each region
    for region, reco_hits in filtered_RECO_hits_by_region.items():
        print(f"{region} - RECO_hits:")
        display(reco_hits)

    for region, mc_tracks in filtered_MC_tracks_by_region.items():
        print(f"{region} - MC_tracks:")
        display(mc_tracks)

    for region, part_info in filtered_part_info_by_region.items():
        print(f"{region} - part_info:")
        display(part_info)

Region_1: (0, 400):


,event,particle_id,hit_id,X,Y,Z,time,E,label,npeak
0,12300006,32,0,36.513824,-125.199722,105.147636,2.834566,0.000001,ACTIVE,1
1,12300006,31,0,36.547150,-125.759644,104.332382,2.629001,0.002518,ACTIVE,1
2,12300006,31,1,36.605511,-126.437218,103.599846,2.633066,0.002213,ACTIVE,1
3,12300006,31,2,36.760361,-127.064156,102.854980,2.637081,0.002263,ACTIVE,1
4,12300006,31,3,37.286903,-127.554703,102.174568,2.641119,0.000403,ACTIVE,1
...,...,...,...,...,...,...,...,...,...,...
503876,12204084,21,267,-171.579620,-160.959534,42.640835,3.820733,0.001174,ACTIVE,1
503877,12204084,21,268,-171.586945,-160.959290,42.638660,3.820999,0.000308,ACTIVE,1
503878,12204084,21,269,-171.587143,-160.957260,42.640736,3.821089,0.000064,ACTIVE,1
503879,12204084,21,270,-171.589188,-160.950531,42.644882,3.821345,0.002224,ACTIVE,1


Region_1: (0, 400) - RECO_hits:


,event,time,npeak,Xpeak,Ypeak,nsipm,X,Y,Xrms,Yrms,Z,Q,E,Qc,Ec,track_id,Ep
0,12300006,2.460001e+13,0,6.515505,-148.908484,1,-19.125,-155.375,0.0,0.0,40.263651,7.188406,574.714427,-1.0,0.001402,-1,-1.0
1,12300006,2.460001e+13,0,6.515505,-148.908484,1,-19.125,-139.825,0.0,0.0,40.263651,5.217391,417.131443,-1.0,0.001018,-1,-1.0
2,12300006,2.460001e+13,0,6.515505,-148.908484,1,-3.575,-155.375,0.0,0.0,40.263651,15.072464,1205.046364,-1.0,0.002943,-1,-1.0
3,12300006,2.460001e+13,0,6.515505,-148.908484,1,-3.575,-139.825,0.0,0.0,40.263651,9.449275,755.471336,-1.0,0.001847,-1,-1.0
4,12300006,2.460001e+13,0,6.515505,-148.908484,1,10.975,-155.375,0.0,0.0,40.263651,8.811594,704.488618,-1.0,0.001721,-1,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420021,12204084,2.440817e+13,0,-139.052600,-162.959196,1,-19.125,-340.975,0.0,0.0,222.271624,10.086956,402.792083,-1.0,0.001039,-1,-1.0
420022,12204084,2.440817e+13,0,-139.052600,-162.959196,1,-19.125,-325.425,0.0,0.0,222.271624,7.478261,298.621737,-1.0,0.000761,-1,-1.0
420023,12204084,2.440817e+13,0,-139.052600,-162.959196,1,-50.225,-356.525,0.0,0.0,225.540513,9.159420,956.310386,-1.0,0.002474,-1,-1.0
420024,12204084,2.440817e+13,0,-139.052600,-162.959196,1,-50.225,-340.975,0.0,0.0,225.540513,9.217391,962.362982,-1.0,0.002482,-1,-1.0


Region_2: (401, 800) - RECO_hits:


...

,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
4393,12060049,1,Tl208,True,0,398.796722,401.564789,564.256775,0.000000,398.796722,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,none,RadioactiveDecay
4394,12060049,4,e-,False,1,398.796722,401.564789,564.256775,0.005479,399.073914,...,1.456231,0.522379,-0.118311,0.000000,0.00000,0.000000,1.122587,0.816352,RadioactiveDecay,NoProcess
4395,12060049,3,anti_nu_e,False,1,398.796722,401.564789,564.256775,0.005479,-12500.000000,...,-0.319769,-0.171160,0.171528,-0.319769,-0.17116,0.171528,0.401211,16183.966797,RadioactiveDecay,Transportation
4396,12060049,2,Pb208[3475.078],False,1,398.796722,401.564789,564.256775,0.005479,398.796722,...,-1.136462,-0.351219,-0.053217,-0.000000,-0.00000,-0.000000,0.000004,0.000000,RadioactiveDecay,RadioactiveDecay
4397,12060049,6,gamma,False,2,398.796722,401.564789,564.256775,0.005479,385.813232,...,-0.724063,0.192750,0.423242,0.000000,-0.00000,-0.000000,0.860554,187.779343,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46735,12276106,56596,e-,False,28,11.984161,288.144958,598.536194,2.042380,11.999187,...,0.028514,-0.093333,0.067202,0.000000,0.00000,0.000000,0.013558,0.623786,eIoni,NoProcess
46736,12276106,64490,e-,False,28,25.183088,290.530518,600.848267,2.281452,25.521288,...,0.061537,-0.134107,-0.074050,0.000000,0.00000,0.000000,0.026007,1.337588,eIoni,NoProcess
46737,12276106,68693,e-,False,28,35.879627,284.074799,602.203918,2.356545,35.862411,...,0.120873,-0.113100,-0.149247,0.000000,-0.00000,0.000000,0.046492,4.326399,eIoni,NoProcess
46738,12276106,28,e-,False,27,228.584320,258.353027,565.041565,0.741902,33.054043,...,-0.860964,-1.165000,-1.093870,0.000000,-0.00000,-0.000000,1.374779,455.185272,conv,NoProcess


Region_3: (801, 1200) - part_info:


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
9241,12096083,1,Tl208,True,0,398.290588,401.510193,563.945374,0.000000,398.290588,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,none,RadioactiveDecay
9242,12096083,4,e-,False,1,398.290588,401.510193,563.945374,0.000913,398.287842,...,-0.161429,-0.581409,-0.239391,0.000000,-0.000000,0.000000,0.315151,0.193766,RadioactiveDecay,NoProcess
9243,12096083,3,anti_nu_e,False,1,398.290588,401.510193,563.945374,0.000913,1111.551270,...,0.082538,1.400028,0.491277,0.082538,1.400028,0.491277,1.486016,12841.565430,RadioactiveDecay,Transportation
9244,12096083,2,Pb208[3197.711],False,1,398.290588,401.510193,563.945374,0.000913,398.290588,...,0.078891,-0.818619,-0.251886,0.000000,-0.000000,-0.000000,0.000002,0.000000,RadioactiveDecay,RadioactiveDecay
9245,12096083,6,gamma,False,2,398.290588,401.510193,563.945374,0.000913,423.187042,...,-0.535708,0.075055,0.217927,0.000000,0.000000,0.000000,0.583188,127.054665,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19959,12312111,38518,e-,False,32,172.627258,154.543106,793.357300,1.831391,173.548813,...,0.050796,0.089727,-0.290394,-0.000000,0.000000,0.000000,0.085725,9.749102,eIoni,NoProcess
19960,12312111,53448,e-,False,32,219.211700,99.365822,785.203369,2.351288,219.560486,...,0.098583,0.053446,-0.111036,0.000000,-0.000000,0.000000,0.023813,1.104148,eIoni,NoProcess
19961,12312111,54697,e-,False,32,219.542618,96.387474,784.365723,2.363810,219.037674,...,-0.119655,-0.234418,0.029191,-0.000000,0.000000,-0.000000,0.064537,7.351887,eIoni,NoProcess
19962,12312111,32,e-,False,31,208.782532,236.079941,779.058533,1.104853,227.313126,...,-1.571687,-0.292809,0.476630,0.000000,-0.000000,0.000000,1.233774,472.787018,conv,NoProcess


Region_4: (1201, 1600) - part_info:


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,...,initial_momentum_x,initial_momentum_y,initial_momentum_z,final_momentum_x,final_momentum_y,final_momentum_z,kin_energy,length,creator_proc,final_proc
4393,12060049,1,Tl208,True,0,398.796722,401.564789,564.256775,0.000000,398.796722,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,none,RadioactiveDecay
4394,12060049,4,e-,False,1,398.796722,401.564789,564.256775,0.005479,399.073914,...,1.456231,0.522379,-0.118311,0.000000,0.00000,0.000000,1.122587,0.816352,RadioactiveDecay,NoProcess
4395,12060049,3,anti_nu_e,False,1,398.796722,401.564789,564.256775,0.005479,-12500.000000,...,-0.319769,-0.171160,0.171528,-0.319769,-0.17116,0.171528,0.401211,16183.966797,RadioactiveDecay,Transportation
4396,12060049,2,Pb208[3475.078],False,1,398.796722,401.564789,564.256775,0.005479,398.796722,...,-1.136462,-0.351219,-0.053217,-0.000000,-0.00000,-0.000000,0.000004,0.000000,RadioactiveDecay,RadioactiveDecay
4397,12060049,6,gamma,False,2,398.796722,401.564789,564.256775,0.005479,385.813232,...,-0.724063,0.192750,0.423242,0.000000,-0.00000,-0.000000,0.860554,187.779343,RadioactiveDecay,phot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46735,12276106,56596,e-,False,28,11.984161,288.144958,598.536194,2.042380,11.999187,...,0.028514,-0.093333,0.067202,0.000000,0.00000,0.000000,0.013558,0.623786,eIoni,NoProcess
46736,12276106,64490,e-,False,28,25.183088,290.530518,600.848267,2.281452,25.521288,...,0.061537,-0.134107,-0.074050,0.000000,0.00000,0.000000,0.026007,1.337588,eIoni,NoProcess
46737,12276106,68693,e-,False,28,35.879627,284.074799,602.203918,2.356545,35.862411,...,0.120873,-0.113100,-0.149247,0.000000,-0.00000,0.000000,0.046492,4.326399,eIoni,NoProcess
46738,12276106,28,e-,False,27,228.584320,258.353027,565.041565,0.741902,33.054043,...,-0.860964,-1.165000,-1.093870,0.000000,-0.00000,-0.000000,1.374779,455.185272,conv,NoProcess


In [36]:
output_dir = '../data/CLEAN/regionalised'
os.makedirs(output_dir, exist_ok=True)

for i, (z_min, z_max) in enumerate(z_regions):
    region_key = f"Region_{i+1}: {z_min, z_max}"
    output_file = f'{output_dir}/MCTl_208_{z_min}-{z_max}mm.h5'
    
    with tb.open_file(output_file, 'w') as h5out:
        df_writer(h5out, filtered_part_info_by_region[region_key], 'MC', 'particles')
        df_writer(h5out, filtered_RECO_hits_by_region[region_key], 'RECO', 'Events')
        df_writer(h5out, filtered_events_by_region[region_key], 'MC', 'hits')
        df_writer(h5out, filtered_MC_tracks_by_region[region_key], 'Tracking', 'MC_Tracks')
    